## Working with categorical data

https://app.datacamp.com/learn/courses/working-with-categorical-data-in-python




In [1]:
import pandas as pd
import numpy as np

df_adults = pd.read_csv('./data/adult.csv')
df_adults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Age              32561 non-null  int64 
 1   Workclass        32561 non-null  object
 2   fnlgwt           32561 non-null  int64 
 3   Education        32561 non-null  object
 4   Education Num    32561 non-null  int64 
 5   Marital Status   32561 non-null  object
 6   Occupation       32561 non-null  object
 7   Relationship     32561 non-null  object
 8   Race             32561 non-null  object
 9   Sex              32561 non-null  object
 10  Capital Gain     32561 non-null  int64 
 11  Capital Loss     32561 non-null  int64 
 12  Hours/Week       32561 non-null  int64 
 13  Country          32561 non-null  object
 14  Above/Below 50k  32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [2]:
df_adults["Marital Status"].describe()  # 7 levels/cat's

count                   32561
unique                      7
top        Married-civ-spouse
freq                    14976
Name: Marital Status, dtype: object

In [3]:
df_adults["Marital Status"].value_counts()  # raw counts per level

Marital Status
Married-civ-spouse       14976
Never-married            10683
Divorced                  4443
Separated                 1025
Widowed                    993
Married-spouse-absent      418
Married-AF-spouse           23
Name: count, dtype: int64

In [4]:
df_adults["Marital Status"].value_counts(normalize=True)  # relative frequencies

Marital Status
Married-civ-spouse       0.459937
Never-married            0.328092
Divorced                 0.136452
Separated                0.031479
Widowed                  0.030497
Married-spouse-absent    0.012837
Married-AF-spouse        0.000706
Name: proportion, dtype: float64

In [5]:
# Explore the Above/Below 50k variable
print(df_adults["Above/Below 50k"].describe(), "\n")

# Print a frequency table of "Above/Below 50k"
print(df_adults["Above/Below 50k"].value_counts(), "\n")

# Print relative frequency values
print(df_adults["Above/Below 50k"].value_counts(normalize=True))

count      32561
unique         2
top        <=50K
freq       24720
Name: Above/Below 50k, dtype: object 

Above/Below 50k
<=50K    24720
>50K      7841
Name: count, dtype: int64 

Above/Below 50k
<=50K    0.75919
>50K     0.24081
Name: proportion, dtype: float64


In [6]:
df_adults.dtypes

Age                 int64
Workclass          object
fnlgwt              int64
Education          object
Education Num       int64
Marital Status     object
Occupation         object
Relationship       object
Race               object
Sex                object
Capital Gain        int64
Capital Loss        int64
Hours/Week          int64
Country            object
Above/Below 50k    object
dtype: object

In [7]:
print(df_adults["Marital Status"].dtype)  # NOTE sigular dtype because working with a series (not a dataframe), 'O' = object
print(df_adults["Marital Status"].nbytes)  #

object
260488


In [8]:
# convert to categorical type
df_adults["Marital Status"] = df_adults["Marital Status"].astype("category")
print(df_adults["Marital Status"].dtype)  # displayes the levels
                                          # "ordered=False" because no order has been specified
print(df_adults["Marital Status"].nbytes)  # big memor savings! 260488 >> 32617

category
32617


## Creating a Categorical Series

+ series that do **NOT** have a logical order
+ series that **DO** have a logical order

## Why use Categorical dtype?

+ big memory saver - compare previous 2 cells above

## Best practice of setting dtypes

If you know the `dtype` of the columns you are reading in advance, it is best practice to specify them in the read.

In [9]:
my_data = ["A", "A", "C", "B", "C", "A"]
# if series does not have a logical order
my_series1 = pd.Series(my_data, dtype="category")
print(my_series1)

0    A
1    A
2    C
3    B
4    C
5    A
dtype: category
Categories (3, object): ['A', 'B', 'C']


In [10]:
# if series does have a logical order (ordinal)
my_series2 = pd.Categorical(my_data, categories=["C", "B", "A"], ordered=True)  # lowest to highest: 'C' < 'B' < 'A'
print(my_series2)

['A', 'A', 'C', 'B', 'C', 'A']
Categories (3, object): ['C' < 'B' < 'A']


In [11]:
# specify dtype if you know in advance
adult_types = {"Marital Status" : "category"}
df_adult = pd.read_csv("./data/adult.csv", dtype=adult_types)
print(df_adult["Marital Status"].dtype)

category


## Grouping data by category

<code>
adult = pd.read_csv("data/adult.csv")

adult1 = adult[adult["Above/Below 50k"] == " <=50K"] 
adult2 = adult[adult["Above/Below 50k"] == " >50K"]
</code>

The last two lines can replace by a one-liner using `group_by`

In [12]:
groupby_object = df_adult.groupby(by=["Above/Below 50k"])

groupby_object.mean(numeric_only=True)  # remember to specify numeric_only parameter (course code is incorrect for newer versions of pandas)

,Age,fnlgwt,Education Num,Capital Gain,Capital Loss,Hours/Week
Above/Below 50k,,,,,,
<=50K,36.783738,190340.86517,9.595065,148.752468,53.142921,38.840210
>50K,44.249841,188005.00000,11.611657,4006.142456,195.001530,45.473026


In [13]:
df_adult.groupby(by=["Above/Below 50k"])[['Age', 'Education Num']].count()

,Age,Education Num
Above/Below 50k,,
<=50K,24720,24720
>50K,7841,7841


In [14]:
# option 1: specify column, especially on large datasets (course code is incorrect for newer versions of pandas)
df_adult.groupby(by=["Above/Below 50k"])[['Age', 'Education Num']].sum()

,Age,Education Num
Above/Below 50k,,
<=50K,909294,237190
>50K,346963,91047


In [15]:
# option 2: course code leaves out "numeric_only=True"
df_adult.groupby(by=["Above/Below 50k"]).sum(numeric_only=True)[['Age', 'Education Num']] 

,Age,Education Num
Above/Below 50k,,
<=50K,909294,237190
>50K,346963,91047


## Be mindful of the order of calculation

In the above 2 examples, option 1 tends to be much faster because with option 2, you're doing the computation on all columns and then displaying the 2 you want (`Age` and `Education Num`). With option 1, you are calculating on just the columns you want to display.

## Groupby on multiple columns

Specifying more than one column to `group_by` creates groups with all the combination of the specified columns, so be careful here as well.

In [16]:
df_adult.groupby(by=["Above/Below 50k", "Marital Status"], observed=False).size()  # looks like all combinations are observed
# NOTICE:  Married-AF-spouse only had 10 instances!

Above/Below 50k  Marital Status       
<=50K            Divorced                  3980
                 Married-AF-spouse           13
                 Married-civ-spouse        8284
                 Married-spouse-absent      384
                 Never-married            10192
                 Separated                  959
                 Widowed                    908
>50K             Divorced                   463
                 Married-AF-spouse           10
                 Married-civ-spouse        6692
                 Married-spouse-absent       34
                 Never-married              491
                 Separated                   66
                 Widowed                     85
dtype: int64

## 

In [17]:
# Group the adult dataset by "Sex" and "Above/Below 50k"
gb = df_adult.groupby(by=["Sex", "Above/Below 50k"])

# Print out how many rows are in each created group
print(gb.size())

Sex     Above/Below 50k
Female  <=50K               9592
        >50K                1179
Male    <=50K              15128
        >50K                6662
dtype: int64


In [18]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html
# margins parameter displays totals of rows and columns of contingency table
pd.crosstab(df_adult["Sex"], df_adult["Above/Below 50k"], margins=True)

Above/Below 50k,<=50K,>50K,All
Sex,,,
Female,9592,1179,10771
Male,15128,6662,21790
All,24720,7841,32561


In [19]:
pd.crosstab(df_adult["Sex"], df_adult["Above/Below 50k"], normalize=True)

Above/Below 50k,<=50K,>50K
Sex,,
Female,0.294586,0.036209
Male,0.464605,0.204601


In [20]:
# Print out the mean of each group for all columns
print(gb.mean(numeric_only=True))

                              Age         fnlgwt  Education Num  Capital Gain  \
Sex    Above/Below 50k                                                          
Female <=50K            36.210801  185999.381359       9.820475    121.986134   
       >50K             42.125530  183687.406277      11.787108   4200.389313   
Male   <=50K            37.147012  193093.609268       9.452142    165.723823   
       >50K             44.625788  188769.101321      11.580606   3971.765836   

                        Capital Loss  Hours/Week  
Sex    Above/Below 50k                            
Female <=50K               47.364470   35.916701  
       >50K               173.648855   40.426633  
Male   <=50K               56.806782   40.693879  
       >50K               198.780396   46.366106  


## Using pandas functions effectively

To efficiently compute a statistic (e.g. mean) for each group in a dataframe:

1. create the grouping
2. compute the statistic on the variable of interest for that group

An example of effiently computing the mean hours per week for each ("Education", "Above/Below 50k") grouping is shown in the next cell.

### The inefficient way...

The **inefficient** way to do the same thing would be to compute the means of all the numeric variables by the grouping and then filter for the variable we want. This would be done by substituting the following line for the last line of in the `code` cell that follows:  
<code>
df_adult.groupby(by=["Education", "Above/Below 50k"]).mean()[["Hours/Week"]] 
</code>



In [21]:
# Create a list of user-selected variables
user_list = ["Education", "Above/Below 50k"]

# Create a GroupBy object using this list
gb = df_adult.groupby(by=user_list)

# Find the mean for the variable "Hours/Week" for each group efficiently
# i.e. only compute the mean after filtering for the data we are interested in
print(gb["Hours/Week"].mean())

Education     Above/Below 50k
10th          <=50K              36.574053
              >50K               43.774194
11th          <=50K              33.322870
              >50K               45.133333
12th          <=50K              35.035000
              >50K               44.818182
1st-4th       <=50K              37.864198
              >50K               48.833333
5th-6th       <=50K              38.539432
              >50K               46.000000
7th-8th       <=50K              38.830033
              >50K               47.500000
9th           <=50K              37.667351
              >50K               44.851852
Assoc-acdm    <=50K              39.264339
              >50K               44.256604
Assoc-voc     <=50K              40.817826
              >50K               43.853186
Bachelors     <=50K              40.586152
              >50K               45.475462
Doctorate     <=50K              45.429907
              >50K               47.513072
HS-grad       <=50K     

## Setting category variables

The `.cat` accessor objects let's us access and manipulate the categories of a categorical `Series`. They take the form `Series.cat.method_name`. Common parameters to pass these methods are:

+ new_categories - a list of categories
+ inples: Boolean - whether or not the update should overwrite the Series
+ ordered: Boolean - whether or not the categorical is treated as an orderd categorical

## Categorical Methods

+ `cat.set_categories()` - Can be used to set the order of categories. All values not specified in this method are dropped
+ `cat.add_categories()` - Does not change the value of any data in the DataFrame. Categories not listed in this method are left alone
+ `cat.remove_categories` -  Values matching categories listed are set to `NaN`

In [22]:
df_dogs = pd.read_csv("./data/ShelterDogs.csv")
df_dogs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2937 entries, 0 to 2936
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 2937 non-null   int64  
 1   name               2845 non-null   object 
 2   age                2937 non-null   float64
 3   sex                2937 non-null   object 
 4   breed              2937 non-null   object 
 5   date_found         2937 non-null   object 
 6   adoptable_from     2937 non-null   object 
 7   posted             2937 non-null   object 
 8   color              2937 non-null   object 
 9   coat               2937 non-null   object 
 10  size               2937 non-null   object 
 11  neutered           1852 non-null   object 
 12  housebroken        460 non-null    object 
 13  likes_people       1999 non-null   object 
 14  likes_children     1219 non-null   object 
 15  get_along_males    1633 non-null   object 
 16  get_along_females  1673 

In [23]:
# convert coat to a categorical variable and look at frequency distribution
df_dogs["coat"] = df_dogs["coat"].astype("category")
df_dogs["coat"].value_counts(dropna=False)  # passing dropna=False to check for missing values

coat
short         1972
medium         565
wirehaired     220
long           180
Name: count, dtype: int64

In [24]:
df_dogs["coat"] = df_dogs["coat"].cat.set_categories(
    new_categories = ["short", "medium", "long"],  # hair length
    ordered = True
)
# drop the values that aren't one of our specified categories above
df_dogs["coat"].value_counts(dropna=False)

coat
short     1972
medium     565
NaN        220
long       180
Name: count, dtype: int64

In [25]:
df_dogs["coat"].head()

0     short
1     short
2     short
3    medium
4     short
Name: coat, dtype: category
Categories (3, object): ['short' < 'medium' < 'long']

In [26]:
df_dogs["likes_people"].value_counts(dropna=False)

likes_people
yes    1991
NaN     938
no        8
Name: count, dtype: int64

In [27]:
# add category that maps to NaN
df_dogs["likes_people"] = df_dogs["likes_people"].astype("category")
df_dogs["likes_people"] = df_dogs["likes_people"].cat.add_categories(
    new_categories=["did not check", "could not tell"]
)
df_dogs["likes_people"].cat.categories

Index(['no', 'yes', 'did not check', 'could not tell'], dtype='object')

In [28]:
# we've created the new categories, but have not set any values to these new categories
df_dogs["likes_people"].value_counts(dropna=False)

likes_people
yes               1991
NaN                938
no                   8
did not check        0
could not tell       0
Name: count, dtype: int64

In [29]:
# removing categories sets them to NaN
df_dogs = pd.read_csv("./data/ShelterDogs.csv")  # reread so we can drop wirehaired category
df_dogs["coat"] = df_dogs["coat"].astype("category")
df_dogs["coat"] = df_dogs["coat"].cat.remove_categories(removals=["wirehaired"])
df_dogs["coat"].cat.categories

Index(['long', 'medium', 'short'], dtype='object')